# **2022 ML FALL HW1: PM2.5 Prediction (Regression)**

Author: MLTAs

Methods:
* Training with all data
* Training config: mini-batch=512, optimizer=Adam, learning rate=0.1 (TODO: Change the config!)



# **Import Some Packages**

In [ ]:
import numpy as np
import csv
import math
import pandas as pd

In [ ]:
import os
os.getcwd()

'/content'

# **Fix random seed**


This is for the reproduction of your result. **DO NOT modify this secton!** 


In [ ]:
import random

seed = 9487
np.random.seed(seed)

# **Download training data**


In [ ]:
!gdown --id "1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy" --output "train.csv"
!gdown --id '155N6fzI7vAFzHAGdy6jkaWIksWH6Y1G2' --output "test.csv"

# Incase the links above die, you can use the following instead.
#!gdown --id '11abE854Eyv4BA7qt5k8r_80sJ3KuOQUN' --output "train.csv"
#!gdown --id '1uod-Z4ztluXnuHtgUbm39nMudUKqXHMl' --output "test.csv"

# If the data is still missing, you can manually download it from kaggle, and upload the files under /content

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy
To: /content/train.csv
100% 324k/324k [00:00<00:00, 78.0MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=155N6fzI7vAFzHAGdy6jkaWIksWH6Y1G2
To: /content/test.csv
100% 49.0k/49.0k [00:00<00:00, 54.0MB/s]


In [ ]:
def valid(x, y):
  # TODO: Try to filter out extreme values.
  #  ex: If PM2.5 > 100, then we don't use the data to train (return False), otherwise return True,

  return y < 24
  #return True

# Create your dataset
def parse2train(data, feats):

  x = []
  y = []
  timestep = 8
  # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
  total_length = data.shape[1] - timestep

  for i in range(total_length):
    x_tmp = data[feats, i:i+timestep] # Use data #0~#7 to predict #8, data #1~#8 to predict #9, etc.
    y_tmp = data[-1, i+timestep] # last column of (i+8)th row: PM2.5

    # Filter out extreme values to train.
    if valid(x_tmp, y_tmp):
      x.append(x_tmp.reshape(-1,))
      y.append(y_tmp)
  
  # x.shape: (n, 15, 8)
  # y.shape: (n, 1)
  #x = x[:,:,1:7] 
  
  x = np.array(x)
  
  y = np.array(y)
  return x,y


#**Adam**
* This is our gradient descent algorithm. Adam was implemented.
* You can implement another algorithm such as SGD, which may (or may not) boost the performance.
* However, **modules like sklearn and pytorch are not allowed**.
* Ref: https://arxiv.org/pdf/1412.6980.pdf

![](https://i.imgur.com/jRaebdf.png)



In [ ]:
# TODO: Implement 2-nd polynomial regression version for the report.
def minibatch(x, y, config):
  
    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    
    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    lam = config.lam
    epoch = config.epoch

    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    # Linear regression: only contains two parameters (w, b).
    w_0 = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    
    # Training loop
    for num in range(epoch):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b*batch_size:(b+1)*batch_size]
            y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)

            # Prediction of linear regression 
            pred = np.dot(x_batch,w_0) + bias
            # loss
            loss = y_batch - pred
            
            # Compute gradient
            g_t = np.dot(x_batch.transpose(),loss) * (-2) +  2 * lam * np.sum(w_0) 
            g_t_b = loss.sum(axis=0) * (-2)
            m_t = beta_1*m_t + (1-beta_1)*g_t 
            v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
            m_cap = m_t/(1-(beta_1**t))
            v_cap = v_t/(1-(beta_2**t))
            m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
            v_t_b = 0.99*v_t_b + (1-0.99)*(g_t_b*g_t_b) 
            m_cap_b = m_t_b/(1-(0.9**t))
            v_cap_b = v_t_b/(1-(0.99**t))
            w_0 = np.copy(w_0)
            
            # Update weight & bias
            w_0 -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
            bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)
            

    return w_0, bias

In [ ]:
def secondminibatch(x, y, config):
  
    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    
    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    lam = config.lam
    epoch = config.epoch

    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    # Linear regression: only contains two parameters (w, b).
    w_0 = np.full(x[0].shape, 0.1).reshape(-1, 1)
    w_1 = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_2 = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t_2 = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    
    # Training loop
    for num in range(epoch):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b*batch_size:(b+1)*batch_size]
            y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)

            # Prediction of linear regression 
            x_square = np.power(x_batch,2)
            pred = np.dot(x_square,w_1)+np.dot(x_batch,w_0) + bias
            # loss
            loss = y_batch - pred  
            # Compute gradient
            g_t = np.dot(x_batch.transpose(),loss) * (-2) +  2 * lam * np.sum(w_0) 
            g_t_2 = np.dot(x_square.transpose(),loss) * (-2) +  2 * lam * np.sum(w_1)
            g_t_b = loss.sum(axis=0) * (-2)
            m_t = beta_1*m_t + (1-beta_1)*g_t 
            v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
            m_cap = m_t/(1-(beta_1**t))
            v_cap = v_t/(1-(beta_2**t))
            m_t_2 = beta_1*m_t_2 + (1-beta_1)*g_t_2 
            v_t_2 = beta_2*v_t_2 + (1-beta_2)*np.multiply(g_t_2, g_t_2)
            m_cap_2 = m_t_2/(1-(beta_1**t))
            v_cap_2 = v_t_2/(1-(beta_2**t))
            m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
            v_t_b = 0.99*v_t_b + (1-0.99)*(g_t_b*g_t_b) 
            m_cap_b = m_t_b/(1-(0.9**t))
            v_cap_b = v_t_b/(1-(0.99**t))
            
            # Update weight & bias
            w_0 -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
            w_1 -= ((lr*m_cap_2)/(np.sqrt(v_cap_2)+epsilon)).reshape(-1, 1)
            bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)
            

    return w_0,w_1,bias

In [ ]:
from argparse import Namespace

# TODO: Tune the config to boost your performance. 
train_config = Namespace(
    batch_size = 512,
    lr = 0.1,
    lam = 0.001,
    epoch = 1000,
)


# **Training your regression model**

In [ ]:
data = pd.read_csv("/content/train.csv")
data

,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,WS_HR,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,PM2.5
0,10.8,0.32,1.7,8.6,10.3,22.9,21,0.6,0.0,71,1.9,172,171,0.6,15
1,10.8,0.27,1.6,6.2,7.8,23.8,20,1.4,0.0,71,1.7,161,129,1.8,13
2,11.0,0.25,0.9,5.4,6.3,27.4,21,0.8,0.0,68,1.6,152,147,1.5,12
3,11.0,0.23,0.7,3.1,3.8,29.5,21,1.8,0.0,68,1.6,138,145,1.7,9
4,11.3,0.22,0.8,2.9,3.8,30.7,16,1.9,0.0,67,1.6,140,139,1.7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5769,29.0,0.41,1.2,14.0,15.3,23.0,21,1.4,0.0,74,2.8,149,168,2.0,14
5770,28.2,0.33,1.7,11.7,13.5,19.5,23,2.1,0.0,78,2.3,187,179,2.5,15
5771,28.0,0.29,1.3,9.1,10.4,17.6,17,1.5,0.0,78,2.0,173,200,1.5,13
5772,28.0,0.27,1.4,9.5,11.0,15.4,17,1.1,0.0,75,1.8,171,135,0.9,10


In [ ]:
# Choose your features to train. 
# Hint: 
# 1. You can select more than one feature.
# 2. You should select "good" features.

# TODO: Carefully justify which feature should be chosen.
#feats = [1,3, 4, 6,10,14]
feats = [2]

In [ ]:
data.corr()


,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,WS_HR,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,PM2.5
AMB_TEMP,1.000000,-0.326709,-0.109779,-0.235131,-0.222598,0.019325,-0.110940,0.205309,0.017442,-0.366906,-0.025951,0.324054,0.304265,0.286994,-0.176147
CO,-0.326709,1.000000,0.567003,0.871899,0.881431,0.033375,0.461834,-0.331012,-0.047807,0.104529,0.357874,0.058534,0.046879,-0.298241,0.659148
NO,-0.109779,0.567003,1.000000,0.481088,0.726947,-0.228651,0.155247,-0.184678,-0.029342,0.047484,0.142316,0.084218,0.088266,-0.153827,0.227219
NO2,-0.235131,0.871899,0.481088,1.000000,0.951711,-0.028912,0.372175,-0.361824,0.001167,0.110824,0.372911,0.075575,0.069703,-0.318930,0.554274
NOx,-0.222598,0.881431,0.726947,0.951711,1.000000,-0.102790,0.345827,-0.348008,-0.009338,0.103411,0.341988,0.088708,0.085474,-0.303606,0.513650
O3,0.019325,0.033375,-0.228651,-0.028912,-0.102790,1.000000,0.271216,0.207841,0.005402,-0.523265,0.220435,0.044655,-0.006989,0.318160,0.233924
PM10,-0.110940,0.461834,0.155247,0.372175,0.345827,0.271216,1.000000,0.037772,-0.091234,-0.222762,0.299310,0.147256,0.113356,0.041243,0.818868
WS_HR,0.205309,-0.331012,-0.184678,-0.361824,-0.348008,0.207841,0.037772,1.000000,-0.011441,-0.449950,-0.111833,0.048408,0.003702,0.816954,-0.102047
RAINFALL,0.017442,-0.047807,-0.029342,0.001167,-0.009338,0.005402,-0.091234,-0.011441,1.000000,0.171105,-0.031188,0.040101,0.038195,0.009345,-0.060801
RH,-0.366906,0.104529,0.047484,0.110824,0.103411,-0.523265,-0.222762,-0.449950,0.171105,1.000000,-0.106848,-0.124297,-0.095074,-0.538838,-0.081576


In [ ]:
# Training data preprocessing.

data = data.values
train_data = np.transpose(np.array(np.float64(data)))
train_x, train_y = parse2train(train_data, feats)
train_x.shape

(5251, 8)

In [ ]:
train_y.mean(0)

9.468672633784042

In [ ]:
#normtrain_x = (train_x - train_x.mean(0))/(train_x.std(0))
normtrain_x = (train_x - train_x.min(0))/(train_x.max(0) - train_x.min(0))

In [ ]:
# Train your regression model

#w_1, bias = minibatch(train_x, train_y, train_config)
#print(w_1.shape, bias.shape)

In [ ]:
#Train second order regression model

w, w1 ,bias = secondminibatch(train_x, train_y, train_config)
print(w.shape,w1.shape, bias.shape)

(8, 1) (8, 1) (1,)


In [ ]:
print(w)
print(w1)
print(bias)

[[ 0.10409929]
 [ 0.10030793]
 [ 0.09522617]
 [ 0.18743798]
 [ 0.06964309]
 [ 0.05384566]
 [-0.01962609]
 [ 0.21361678]]
[[-0.02580325]
 [-0.02338438]
 [-0.01566199]
 [ 0.0022583 ]
 [-0.01313249]
 [-0.05632472]
 [-0.04253674]
 [-0.01052346]]
[7.48876061]


In [ ]:
trainx_pred = []
for i in range(int(train_x.shape[0])):
      # Prediction of linear regression 
      #prediction = (np.dot(np.reshape(w_1,-1),train_x[i]) + bias)[0]
      prediction = (np.dot(np.reshape(w1,-1),np.power(train_x[i],2))+np.dot(np.reshape(w,-1),train_x[i]) + bias)[0]
      trainx_pred.append(prediction)
trainx_pred = np.array(trainx_pred)
print(trainx_pred.shape)
print(train_y.shape)

score = np.sqrt(np.mean(np.power(trainx_pred-train_y,2)))
print(score)

(5251,)
(5251,)
10.152015960955671


# **Testing:**


In [ ]:
def parse2test(data, feats):
  x = []
  timestep = 8
  for i in range(90):
    x_tmp = data[feats,timestep*i: timestep*i+timestep]
    x.append(x_tmp.reshape(-1,))

  # x.shape: (n, 15, 8)
  x = np.array(x)
  return x


In [ ]:
data = pd.read_csv('test.csv')
data = data.values

test_data = np.transpose(np.array(np.float64(data)))
test_x = parse2test(test_data, feats)
#norm_test_x = (test_x - train_x.mean(0))/(train_x.std(0))
norm_test_x = (test_x - train_x.min(0))/(train_x.max(0) - train_x.min(0))

# **Write result as .csv**

---



In [ ]:
 with open('my_sol.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])
    print(norm_test_x.shape) 
    for i in range(int(norm_test_x.shape[0])):
      # Prediction of linear regression 
      #prediction = (np.dot(np.reshape(w_1,-1),test_x[i]) + bias)[0]
      prediction = (np.dot(np.reshape(w1,-1),np.power(test_x[i],2))+np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]
      writer.writerow([i, prediction])

(90, 8)
